# PEC 2: gráfico Ridgeline

Se cargan las librerías necesarias

In [2]:
import kagglehub as khub
import pandas as pd
from ridgeplot import ridgeplot

Se cargan los datos desde kaggle

In [3]:
path = khub.dataset_download("sudalairajkumar/daily-temperature-of-major-cities")
df = pd.read_csv(path+"/city_temperature.csv")
print(df.info())

C:\Users\JLGon\AppData\Local\Temp\ipykernel_15816\1985251444.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+"/city_temperature.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906327 entries, 0 to 2906326
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Region          object 
 1   Country         object 
 2   State           object 
 3   City            object 
 4   Month           int64  
 5   Day             int64  
 6   Year            int64  
 7   AvgTemperature  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 177.4+ MB
None


In [4]:
columns = ['Month', 'AvgTemperature']

# Se eliminan outliers de temperatura y se mantienen solo las de Barcelona 
df_bcn =  df[(df['City'] == 'Barcelona') & (df['AvgTemperature']>0)][columns]

# Se corrigen las temperaturas con un desfase de 46 para pasarlas a grados Celsius
df_bcn['AvgTemperature'] = df_bcn['AvgTemperature'] - 46
df_bcn['AvgTemperature'] = df_bcn['AvgTemperature'].round(2)
df_bcn.columns = ['Mes', 'Temperatura media diaria (°C)']


# Se resetea el índice
df_bcn.reset_index(drop=True, inplace=True)

df_bcn.head(20)

,Mes,Temperatura media diaria (°C)
0,1,4.8
1,1,-1.5
2,1,-5.0
3,1,-6.8
4,1,-6.2
5,1,-0.8
6,1,-1.2
7,1,-3.6
8,1,4.2
9,1,5.1


In [5]:
# Mostramos cuantos registros hay por mes
df_bcn['Mes'].value_counts().sort_index()

Mes
1     803
2     734
3     805
4     778
5     783
6     746
7     775
8     773
9     748
10    775
11    746
12    770
Name: count, dtype: int64

Se crea el gráfico ridgeline

In [6]:
list_temp = df_bcn.groupby('Mes')['Temperatura media diaria (°C)'].apply(list).to_list()

In [7]:
meses = ['Enero  ',
         'Febrero  ',
         'Marzo  ',
         'Abril  ',
         'Mayo  ',
         'Junio  ',
         'Julio  ',
         'Agosto  ',
         'Septiembre  ',
         'Octubre  ',
         'Noviembre  ',
         'Diciembre  ']

fig = ridgeplot(list_temp,
                bandwidth=2,
                colorscale='RdBu_r',
                colormode='mean-means',
                coloralpha=0.75,
                labels=meses,
                linewidth=2,
                spacing=0.65,
                xpad=0.05)

fig.update_layout(height=1000,
                  width=1400,
                  font_size=15,
                  plot_bgcolor='rgb(240,240,240)',
                  title_text="<b>¿Cómo cambia la temperatura en Barcelona a lo largo del año?</b><br><span style='font-size:15px;'>Distribución de la temperatura media diaria (ºC) por mes en Barcelona  con datos entre los años 1995 y 2020.</span>",
                  title_x=0.09,
                  title_y=0.95,
                  title_font_size=20,
                  title_font_color='black',
                  yaxis_title="Meses del año",
                  xaxis_title="Temperatura media diaria (°C)",
                  margin=dict(t=100, b=150),  # Aumenta el margen inferior
                  annotations=[
                      dict(
                          x=0,
                          y=-0.12,  # Ajusta la posición vertical para el footer
                          xref='paper',
                          yref='paper',
                          text='Source: <a href="https://www.kaggle.com/datasets/sudalairajkumar/daily-temperature-of-major-cities">Kaggle | Daily Temperature of Major Cities</a>',
                          showarrow=False,
                          font=dict(size=15, color='black')
                      ),
                      dict(
                          x=0,
                          y=-0.15,
                          xref='paper',
                          yref='paper',
                          text='Data License: Data files © Original Authors',
                          showarrow=False,
                          font=dict(size=15, color='black')                         
                      )
                  ]
                 )
fig.show()

In [8]:
# Guardamos el gráfico
fig.write_html("ridgeline.html")